## RESUMEN PROYECTO MACHINE LEARNING
## ESPERANZA DE VIDA: PREDICIÓN SIN EDADES
### ADRIAN GARCIA HERNANDEZ

### Obtención de los datos

Los datos han sido obtenidos de la página web del Banco Mundial, más específicamente se trata de los **Indicadores de Desarrollo Mundial**. Presenta los datos de desarrollo global más actuales y precisos disponibles, e incluye estimaciones nacionales, regionales y globales. Los indicadores para cada país son muy numerosos (489 indicadores por país) y con una periodicidad anual que va desde 1960 a 2020. 

La existencia de una gran cantidad de valores nulos ha sido la principal razón por la que se ha cambiado el tipo de análisis inicialmente contemplado. Se han escogido los valores de los años 2000 a 2019, ya que son aquellos que menos valores nulos presentan de toda la serie. Se han anonimizado los valores (eliminado los nombres de los países) y se han añadido por cada país y año una nueva fila que contiene los valores. Por lo tanto, hemos pasado de tener un *panel data* a un *cross-sectional data*. Esto podría suponer un problema ya que realmente los datos que tenemos no se basan en un año en concreto, si no que en realidad están repartidos en un periodo de 20 años. Este problema se puede en parte corregir al hacer uso de variables que no incluyen perturbaciones que no sean incluidos en el dataset, como puede ser la inflación y su efecto en los precios; por ello se hace uso del PIB a precios constantes (US$ a precios de 2015)


El dataset completo puede ser consultado [aqui](https://datacatalog.worldbank.org/search/dataset/0037712/World-Development-Indicators).

### Limpieza de datos & Merging

El dataset estaba estructurado de la siguiente manera, por una parte las columnas describian los años, desde 1960 hasta 2020, por otra las filas en las filas se incluia cada variable posible por cada pais del mundo, resultando en algo más de 380.000 filas. Debido a la gran falta de datos, elegimos únicamente el periodo desde 2000 al 2019, los años con más datos.

Para la conversión de un panel data a un -falso- cross-sectional data:

- Primeramente seleccionamos la primera columna del año escogido asi como las que identifican el país y se le cambia el label del año por 'Value':

    ```python
    columns_to_keep = ['Country Code', 'Indicator Code', '2001']
    df = df.loc[:, columns_to_keep]
    df = df.rename(columns={'2001': 'Value'})
    ```


- Posteriormente pivotamos el dataframe de la siguiente manera:
    ```python
    df = df.pivot(index='Country Code', columns='Indicator Code', values='Value')
    ```


- Reseteamos el índice:
    ```python
    df.reset_index(inplace=True)
    ```

- Guardamos el dataframe resultante en un nuevo csv con el nombre de data y el año que representa los valores:
    ```python
    df.to_csv('data_2001.csv', index=False)
    ```

Una vez contamos con todos los CSV de cada año desde el 2000 al 2019, debemos unirlos en un único dataframe.

Para ello seguimos los siguientes pasos:

- Creamos el path a la carpeta donde se localizan todos los CSV y creamos una lista vacia llamada data_frames:
    ```python
    path = "Data merging/"
    data_frames = []
    ````
- Creamos un bucle for para que recoja todos los archivos CSV dentro de la carpeta y agreagrlos dentro de la lista data_frames:

    ```python
    import os
    for file in os.listdir(path):
        if file.endswith(".csv"):
            df = pd.read_csv(os.path.join(path, file))
            data_frames.append(df)

    ```
- Finalmente usando la funcion concat, unimos todo en un mismo dataframe:
    ```python
    df = pd.concat(data_frames, ignore_index=True)
    ```

- Hacemos limpieza de aquellas columnas y filas que tengan demasiados NaNs:
    ```python
    df = df.dropna(thresh=len(df) - 200, axis=1)

    df = df.dropna(thresh=len(df.columns) - 5, axis=0)
    ```


### Análisis exploratorio & Feature engineering

Pese a deber ser el primer paso a hacer, hasta que no se han hecho todas las transformaciones anteriores no se podia realmente visualizar de manera clara el dataset. Tras estas, se han calculado los principales parámetros, observado la matriz de correlación y ploteado diferentes gráficos para una buena comprensión de los datos que presenta el dataset. 

Al principio se abordó la posibilidad de sustituir los NaNs por la mediana de la columna. Tras comparar los diferentes modelos y comprobar su peor rendimiento se decidió eliminar todos los NaNs. 

Respecto al escalado de los datos, se probaron diferentes formas como por ejemplo, el StandardScaler o el MinMaxScaler, no obstante estos tambien afectaban negatívamente al rendimiento de los modelos. Además, afectaban a la correcta comprensión de del MSE o el R2, haciendo necesario calcular estos su magnitud natural.

### Modelos

Se han probado diversos modelos y calculado sus respectivas métricas para comprobar su rendimiento. 
Se han considerado los siguientes modelos: Linear Regression, Decision Tree, Random Forest, Adaboost, Gradient Boosting, XGBoost (con y sin convertir la variable dependiente a logarítmica). También se han probado modelos de regresión lineal regularizados usando los métodos: ElasticNet, Ridge, Lasso y Bayesian Ridge; y se han buscado los mejores hiperparámetros de estos. Por otra parte, también se ha hecho uso de un pipeline para la normalización de los datos con StandardScaler y la búsqueda de hiperparámetros del modelo KernelRidge usando GridSearchCV.

Finalmente, tras comparar los resultados de cada uno de los modelos se ha determinado que aquel con mayor precisión es el **Random Forest**.

Este ha obtenido un error cuadrático medio igual a 1.53 lo que indica que en promedio, las predicciones del modelo difieren en 1.53 años del valor real de X. Un MSE de 1.53 años es considerado relativamente bajo, lo que sugiere que el modelo puede estar haciendo buenas predicciones en general.

Ha obtenido un coeficiente de determinación(R2) igual a 0.98, esto indica que aproximadamente el 98% de la variabilidad de la variable dependiente(la que predecimos) se puede explicar por la relación con las variables independientes incluidas en el modelo, lo que sugiere que el modelo tiene un buen ajuste a los datos.

Para confirmar la fiabilidad del modelo también se deben de comprobar que los residuos no tengan autocorrelación, heterocedasticidad, no estacionariedad  y que estos sigan una distribución normal.

Para comprobar la autocorrelación se ha llevado a cabo el test de **Durbin Watson**, el cual ha demostrado que no hay una autocorrelación significativa en los residuos. Para comprobar que no haya heterocedasticidad, se ha hecho el test de **Breush-Pagan**, confirmando la homocedasticidad de los residuos. Respecto a la no estacionariedad(no Unit Root) se ha probado el test de **Dickey-Fuller** el cual ha rechazado la hipótesis nula, es decir, que los residuos no tienen Unit Root, o lo que es lo mismo, que los residuos son estacionarios. Por último, se ha comprobado la normalidad en la distribución de los residuos; para ello se han hecho dos tests, el de **Shapiro-Wilk** y el de **Kolmogorov-Smirnov**. Ambos tests han determinado que los residuos **NO** se distribuyen de manera normal. Esto puede suponer un problema ya que se podría estar violando alguna suposición del modelo. No obstante, al tratarse de una muestra muy grande (n>100) podemos hacer uso del **Teorema Central del Límite** y asumir que se distribuyen de una manera aproximadamente Normal.




Aqui debajo se especifica el código del modelo finalmente escogido:


```python
models = {"Random Forest": RandomForestRegressor()}

for name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Evaluate the model's performance
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    # Print the model's performance metrics
    print(f"{name}:")
    print(f"  Mean Squared Error: {mse:.5f}")
    print(f"  Root Mean Squared Error: {rmse:.2f}")
    print(f"  R-squared: {r2:.2f}")

```

Código de los tests realizados:


- **TEST AUTOCORRELACIÓN DURBING-WATSON**

    ```python
    from statsmodels.stats.stattools import durbin_watson

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Calculate the residuals
    residuals = y_test - y_pred

    # Calculate the Durbin-Watson statistic
    durbin_watson_statistic = durbin_watson(residuals)

    # Print the Durbin-Watson statistic
    print("Durbin-Watson statistic:")
    print(f"  {durbin_watson_statistic:.3f}")

    # Interpret the Durbin-Watson statistic
    if durbin_watson_statistic < 1.5:
        print("There may be positive autocorrelation in the residuals.")
    elif durbin_watson_statistic > 2.5:
        print("There may be negative autocorrelation in the residuals.")
    else:
        print("There is no significant autocorrelation in the residuals.")
    ```
- **TEST HETEROCEDASTICIDAD BREUSCH-PAGAN**

    ```python
    from statsmodels.api import add_constant
    from statsmodels.stats.diagnostic import het_breuschpagan

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Calculate the residuals
    residuals = y_test - y_pred

    # Add a constant column to X_test
    X_test_with_const = add_constant(X_test)

    # Calculate the test statistic and p-value for the Breusch-Pagan test
    test_stat, p_value, _, _ = het_breuschpagan(residuals, X_test_with_const)

    # Print the test statistic and p-value
    print("Breusch-Pagan test for homoscedasticity:")
    print(f"  Test Statistic: {test_stat:.3f}")
    print(f"  p-value: {p_value:.3f}")

    # Interpret the test results
    if p_value < 0.05:
        print("The residuals are not homoscedastic.")
    else:
        print("The residuals are homoscedastic.")
    ```
- **TEST NO ESTACIONARIEDAD DICKEY-FULLER**
    ```python
    from statsmodels.tsa.stattools import adfuller

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Calculate the residuals
    residuals = y_test - y_pred

    # Apply the Dickey-Fuller test to the residuals
    adf_test = adfuller(residuals)

    # Print the test statistic and p-value
    print("Dickey-Fuller test:")
    print(f"  Test statistic: {adf_test[0]:.3f}")
    print(f"  p-value: {adf_test[1]:.3f}")

    # Interpret the test result
    if adf_test[1] < 0.05:
        print("Reject the null hypothesis. The residuals do not have a unit root.")
    else:
        print("Fail to reject the null hypothesis. The residuals may have a unit root.")
    ```
- **TEST NORMALIDAD DE LOS RESIDUOS**

    - **Kolmogorov-Smirnov**
        ```python
        from scipy.stats import kstest

        # Calculate the residuals
        residuals = y_test - y_pred

        # Perform the K-S test for normality
        stat, p = kstest(residuals, "norm")

        # Print the test statistic and p-value
        print("Kolmogorov-Smirnov test for normality:")
        print(f"  Test Statistic: {stat:.3f}")
        print(f"  p-value: {p:.3f}")

        # Interpret the test results
        if p < 0.05:
            print("The residuals are not normally distributed.")
        else:
            print("The residuals are normally distributed.")
        ```
    - **Shapiro-Wilk**
        ```python
        from scipy.stats import shapiro

        # Calculate the residuals
        residuals = y_test - y_pred

        # Perform the Shapiro-Wilk test for normality
        stat, p = shapiro(residuals)

        # Print the test statistic and p-value
        print("Shapiro-Wilk test for normality:")
        print(f"  Test Statistic: {stat:.3f}")
        print(f"  p-value: {p:.5f}")

        # Interpret the test results
        if p < 0.005:
            print("The residuals are not normally distributed.")
        else:
            print("The residuals are normally distributed.")
        ```

Tras la elección del modelo (Random forest), se ha procedido a la creación de dor archivos python, preprocessing.py y train.py.

- **preprocessing.py:** Crea una función llamada *load_data* que importa el dataset en un pandas dataframe y selecciona las columnas necesarias para el modelo.

    ```python
    import pandas as pd
    def load_data():
        df = pd.read_csv('/Users/adriangarcia/Desktop/Project_ML_Adri/data/processed/merged_20_years.csv')  
        df = df[['SP.DYN.LE00.IN','NY.GDP.MKTP.KD','SP.POP.TOTL','AG.SRF.TOTL.K2']]
        df = df.dropna()
        return df

    ```
- **train.py:** Llama al archivo *preprocessing.py* para cargar el dataset usando la función *load_data* previamente creada y entrena el modelo, finalmente guardando el modelo en un archivo *new_model.joblib*.

    ```python
    import preprocessing
    import joblib
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.model_selection import train_test_split

    df = preprocessing.load_data()

    X = df[['NY.GDP.MKTP.KD','SP.POP.TOTL','AG.SRF.TOTL.K2']]
    y = df['SP.DYN.LE00.IN']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 42)

    models = {"Random Forest": RandomForestRegressor()}

    for name, model in models.items():
        # Train the model
        model.fit(X_train, y_train)

        # Make predictions on the test set
        y_pred = model.predict(X_test)

    # Train and save the model
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    joblib.dump(model, 'new_model.joblib')

    ```